## 1. Создай соединение с базой данных с помощью библиотеки `sqlite3`.

In [15]:
import sqlite3


sql_conn = sqlite3.connect('../../data/checking-logs.sqlite')
sql_conn

## 2. Создай новую таблицу `datamart` в базе данных, объединив таблицы `pageviews` и `checker` с помощью только **одного** запроса.
- Таблица должна содержать следующие столбцы: `uid`, `labname`, `first_commit_ts`, `first_view_ts`.
- `first_commit_ts` — это просто новое имя для столбца `timestamp` из таблицы `checker`; он показывает первый коммит конкретного лабораторного задания конкретного пользователя.
- `first_view_ts` — первое посещение пользователем из таблицы `pageviews`, метка времени посещения пользователем ленты новостей.
- По-прежнему нужно использовать фильтр `status = 'ready'`.
- По-прежнему нужно использовать фильтр `numTrials = 1`.
- Имена лабораторных заданий по-прежнему должны быть из следующего списка: `laba04`, `laba04s`, `laba05`, `laba06`, `laba06s`, `project1`.
- Таблица должна содержать только пользователей (`uid` с `user_*`), а не администраторов.
- `first_commit_ts` и `first_view_ts` должны быть распарсены как `datetime64[ns]`.

In [17]:
import pandas as pd
pd.read_sql("SELECT c.uid, c.labname, c.timestamp as first_commit_ts, \
                        MIN(p.datetime) as first_view_ts \
                        FROM checker as c \
                        LEFT JOIN pageviews as p \
                        ON c.uid = p.uid \
                        WHERE c.status='ready' AND \
                              c.numTrials=1 AND \
                              c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', \
                                            'laba06s', 'project1') AND \
                              c.uid LIKE 'user_%' \
                        GROUP by c.timestamp, c.uid;",
            sql_conn, parse_dates=['first_commit_ts', 'first_view_ts']).to_sql('datamart', sql_conn, if_exists='replace')
# datamart.to_sql('datamart', sql_conn, if_exists='replace')
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


In [3]:
info = pd.read_sql('PRAGMA table_info(datamart)', sql_conn)
info

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


In [4]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


## 3. Используя методы библиотеки Pandas, создай два датафрейма: `test` и `control`.
- `test` должен включать пользователей, у которых имеются значения в `first_view_ts`.
- `control` должен включать пользователей, у которых отсутствуют значения в `first_view_ts`.
- Замени пропущенные значения в `control` средним значением `first_view_ts` пользователей из `test` (оно пригодится нам для анализа в будущем).
- Сохрани обе таблицы в базе данных (вы будете использовать их в следующих упражнениях).

In [5]:
test = datamart.dropna().copy()
control = datamart[datamart.first_view_ts.isna()].copy()
control.first_view_ts = test.first_view_ts.mean()
test.to_sql('test', sql_conn, if_exists='replace')
control.to_sql('control', sql_conn, if_exists='replace')
test.shape, control.shape

((59, 4), (81, 4))

In [6]:
test.head(3)

,uid,labname,first_commit_ts,first_view_ts
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035


In [7]:
control.head(3)

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783552
1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783552
2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783552


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [9]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 0 to 137
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


## 4. Закрой соединение.

In [10]:
sql_conn.close()

In [11]:
conn = sqlite3.connect('../../data/checking-logs.sqlite')

In [12]:
test = pd.io.sql.read_sql('SELECT * FROM test', conn)

In [13]:
control = pd.io.sql.read_sql('SELECT * FROM control', conn)

In [14]:
conn.close()